In [ ]:
# Author Markus Laubenthal

In [ ]:
!mkdir -p input

In [ ]:
!mkdir -p input/base 
!mkdir -p input/query
!mkdir -p input/ground_truth

In [ ]:
!wget https://aev-autonomous-driving-dataset.s3.eu-central-1.amazonaws.com/a2d2-preview.tar --no-verbose

In [ ]:
!wget https://storage.googleapis.com/laubenthal_spatiolab/kaffeemaschinen.zip -O input/ground_truth/kaffeemaschinen.zip --no-verbose
!wget https://storage.googleapis.com/laubenthal_spatiolab/200x200coffee.zip -O input/ground_truth/200x200xcoffee.zip --no-verbose
!wget https://storage.googleapis.com/laubenthal_spatiolab/horses.zip -O input/ground_truth/horses.zip --no-verbose

In [ ]:
!wget https://storage.googleapis.com/laubenthal_spatiolab/Archiv.zip -O input/base/a2d2_alt.zip --no-verbose

In [ ]:
!unzip input/base/a2d2_alt.zip -d input/base/a2d2_alt

In [ ]:
!tar -xf a2d2-preview.tar -C input/base
!unzip input/ground_truth/kaffeemaschinen.zip -d input/ground_truth/kaffeemaschinen
!unzip input/ground_truth/200x200xcoffee.zip -d input/ground_truth/kaffeemaschinen200
!unzip input/ground_truth/horses.zip -d input/ground_truth/horses

In [ ]:
!rm -rf functions
!git clone https://github.com/Lennard-Alms/lab_bd.git functions

In [ ]:
import tensorflow as tf
import numpy as np
import glob
import cv2
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
import math
import h5py
import keras
from keras.layers import Input
from keras import backend as K
from keras import layers
import gc
from scipy.spatial import distance_matrix
import seaborn as sns
from operator import itemgetter 
from google.colab.patches import cv2_imshow
from functions.preprocessing.BatchToFile import BatchProcessToFile
from functions.preprocessing.FeatureExtractor import VGGFeatureExtractorMax
from functions.preprocessing.FeatureExtractor import get_gem_model
# from functions.preprocessing.FeatureExtracorMaxNoPatches import VGGFeatureExtractorMaxNoPatches

In [ ]:
from functions.preprocessing.ImageMutation import PatchMutation
from functions.preprocessing.HelperFunctions import get_patches_from_image, get_image

In [ ]:
coffeemachines = glob.glob('input/ground_truth/kaffeemaschinen200/*.png')[0:12]
car_images_paths = list(glob.glob('input/base/a2d2_alt/**/*.png', recursive=True))
horses = sorted(glob.glob('input/ground_truth/horses/horses/*.png'))
h5_filename = "max_result.h5"
car_images_paths.sort()

In [ ]:
horse_images = np.array([cv2.imread(horse, cv2.IMREAD_UNCHANGED) for horse in horses])
trans_mask = horse_images[:,:,:,3] == 0

#replace areas of transparency with white and not transparent
_horse_images = horse_images.copy()
_horse_images[trans_mask] = [255, 255, 255, 255]

#new image without alpha channel...
w_horse_images = np.empty((_horse_images.shape[0], _horse_images.shape[1], _horse_images.shape[2], 3)).astype(np.uint8)
for index, whi in enumerate(w_horse_images):
  w_horse_images[index] = cv2.cvtColor(_horse_images[index], cv2.COLOR_BGRA2BGR)
# pm = PatchMutation(horse_images, mutation_probability=1, size=(100,100))

In [ ]:
# Create some Test Images for Report:
pm100 = PatchMutation(horse_images[9:10], mutation_probability=1,size=(170,170))
a2d2_patches = get_patches_from_image(get_image(car_images_paths[0]), window_size=(270,270), window_overlap=0)
query_index = 0
query_image = horse_images[query_index]
db1, label = pm100.mutate(a2d2_patches[np.random.randint(0,20)])
rquery = horse_images
cv2_imshow(cv2.cvtColor(db1, cv2.COLOR_BGR2RGB))


In [ ]:
def get_angle_matrix(feature_vectors, feature_vectors_b = None):
  if feature_vectors_b is None:
    feature_vectors_b = feature_vectors.copy()
  norms = np.linalg.norm(feature_vectors, axis=1)
  norms_b = np.linalg.norm(feature_vectors_b, axis=1)
  angle_matrix = (np.dot(feature_vectors, feature_vectors_b.T) / np.dot(norms[:,np.newaxis], norms_b[np.newaxis, :])).flatten().clip(-1,1)
  angle_matrix = np.arccos(angle_matrix)
  angle_matrix.sort()
  return angle_matrix

In [ ]:
def create_unmutated_dataset(label="unmutated_a2d2"):
  global car_images_paths
  filename="a2d2.h5"
  ci_selection = np.random.randint(0,len(car_images_paths), 76)
  cars = list(itemgetter(*ci_selection)(car_images_paths))
  processor = VGGFeatureExtractorMax(window_size=(200,200), vgg_model=get_gem_model((200,200)))
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, "a2d2", batch_size=12)
# create_unmutated_dataset()

In [ ]:
def create_mutated_dataset(mutation_probability=0.1, label="mutated_a2d2_coffee"):
  global car_images_paths
  global coffeemachines
  global h5_filename
  mutation_strategy = PatchMutation(coffeemachines, mutation_probability=mutation_probability, size=(100,100))
  processor = VGGFeatureExtractorMax(window_size=(200,200), mutation_strategy=mutation_strategy, vgg_model=get_gem_model())
  batch_processor = BatchProcessToFile(h5_filename)
  batch_processor.batch(processor, car_images_paths, label, batch_size=12)

In [ ]:
def create_fully_mutated_dataset(mutation_probability=1, label="mutated_a2d2_coffee_100"):
  global car_images_paths
  global coffeemachines
  global h5_filename
  mutation_strategy = PatchMutation(coffeemachines, mutation_probability=mutation_probability, size=(100,100))
  processor = VGGFeatureExtractorMax(window_size=(200,200), mutation_strategy=mutation_strategy, vgg_model=get_gem_model())
  batch_processor = BatchProcessToFile(h5_filename)
  batch_processor.batch(processor, car_images_paths, label, batch_size=16)

In [ ]:
def create_duplicate_test(mutation_probability=0.1, label="duplicate_test"):
  global horse_images
  global w_horse_images
  global car_images_paths
  global coffeemachines
  global h5_filename
  
  filename="duplicates.h5"
  # Select 10% of dataset
  ci_selection = np.random.randint(0,len(car_images_paths), 76)
  cars = list(itemgetter(*ci_selection)(car_images_paths))

  for window_size in [(50,50), (100,100), (150,150), (200,200), (300,300), (400,400)]:
    query_model = get_gem_model(window_size)
    processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=query_model, extract_patches=False)
    batch_processor = BatchProcessToFile(filename)
    batch_processor.batch(processor, w_horse_images, 'query' + str(window_size), batch_size=128)

  # Create A2D2 images with scaled horses x1
  window_size=(300,300)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(200,200))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_1_0', batch_size=12)

  # Create A2D2 images with scaled horses x1.5
  window_size=(400,400)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(300,300))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_1_5', batch_size=12)

  # Create A2D2 images with scaled horses x2
  window_size=(500,500)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(400,400))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_2_0', batch_size=12)

  # Create A2D2 images with scaled horses x0.5
  window_size=(150,150)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(100,100))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_0_5', batch_size=12)

  # Create A2D2 images with scaled horses x0.25
  window_size=(100,100)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(50,50))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_0_25', batch_size=12)

create_duplicate_test()

In [ ]:
def create_duplicate_test(mutation_probability=0.1, label="duplicate_test"):
  global horse_images
  global w_horse_images
  global car_images_paths
  global coffeemachines
  global h5_filename
  
  filename="duplicates.h5"
  # Select 10% of dataset
  ci_selection = np.random.randint(0,len(car_images_paths), 76)
  cars = list(itemgetter(*ci_selection)(car_images_paths))

  # Create Query images in different sizes

  for window_size in [(100,100), (150,150), (200,200), (300,300), (400,400)]:
    query_model = get_gem_model(window_size)
    processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=query_model, extract_patches=False)
    batch_processor = BatchProcessToFile(filename)
    batch_processor.batch(processor, w_horse_images, 'query' + str(window_size), batch_size=128)

  # Create horse images on white Background
  window_size=(300,300)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=1, size=(200,200))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=False, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, np.full((100, 300, 300, 3), 255), 'white_background_horses', batch_size=128)

  # Create A2D2 images with horses ~50% cover
  window_size=(250,250)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(200,200))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_horses_50_cover', batch_size=12)

  # Create A2D2 images with horses ~40% cover
  window_size=(300,300)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(200,200))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_horses_40_cover', batch_size=12)

  # Create A2D2 images with horses ~20% cover
  window_size=(400,400)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(200,200))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_horses_20_cover', batch_size=12)

  # Create A2D2 images with scaled horses x1.5
  window_size=(400,400)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(300,300))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_1_5', batch_size=12)

  # Create A2D2 images with scaled horses x2
  # filename="test.h5"
  window_size=(500,500)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(400,400))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_2_0', batch_size=12)

  # Create A2D2 images with scaled horses x0.5
  window_size=(150,150)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(100,100))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_0_5', batch_size=12)

  # Create A2D2 images with scaled horses x0.25
  window_size=(100,100)
  vgg_model = get_gem_model(window_size)
  ms = PatchMutation(horse_images, mutation_probability=0.1, size=(50,50))
  processor = VGGFeatureExtractorMax(window_size=window_size, vgg_model=vgg_model, extract_patches=True, mutation_strategy=ms)
  batch_processor = BatchProcessToFile(filename)
  batch_processor.batch(processor, cars, 'a2d2_background_scale_0_25', batch_size=12)
create_duplicate_test()

In [ ]:
with h5py.File('duplicates.h5', 'r') as f:
  for key in f.keys():
    print(key)
    print(f[key].shape)

In [ ]:
def create_white_background_coffeemachines(label="white_coffee", scale=1):
  global coffeemachines
  global h5_filename
  if scale != 1:
    label = label + "_scale_" + str(scale)
  white_images = np.full((500,200,200,3), 255, dtype=np.uint8)
  mutation_strategy = PatchMutation(coffeemachines, mutation_probability=1, size=(int(scale * 100),int(scale * 100)))
  processor = VGGFeatureExtractorMax(window_size=(200,200), mutation_strategy=mutation_strategy, extract_patches=False, vgg_model=get_gem_model())
  batch_processor = BatchProcessToFile(h5_filename)
  batch_processor.batch(processor, white_images, label, batch_size=500)

In [ ]:
def create_original_coffeemachines(label="original_coffeemachines"):
  global coffeemachines
  global h5_filename
  with h5py.File(h5_filename, 'a') as f:
    if label in f:
      del f[label]
  mutation_strategy = None
  processor = VGGFeatureExtractorMax(window_size=(200,200), mutation_strategy=mutation_strategy, extract_patches=False, vgg_model=get_gem_model())
  batch_processor = BatchProcessToFile(h5_filename)
  batch_processor.batch(processor, coffeemachines, label, batch_size=32)

  with h5py.File(h5_filename, 'a') as f:
    dataset_name = label + "_label"
    if dataset_name in f:
      del f[dataset_name]
    f.create_dataset(dataset_name, (len(coffeemachines), 1))
    dataset = f[dataset_name]
    for i, cm in enumerate(coffeemachines):
      dataset[i] = np.array([i])



In [ ]:
create_original_coffeemachines()
create_white_background_coffeemachines()
create_white_background_coffeemachines(scale=0.5)
create_white_background_coffeemachines(scale=0.25)
create_white_background_coffeemachines(scale=1.5)
create_fully_mutated_dataset()
create_mutated_dataset()

In [ ]:
def remove_if_exists(f, key):
  if key in f:
    del f[key]
with h5py.File('max_result.h5', 'r') as f:
  remove_if_exists(f, 'unmutated_a2d2')

In [ ]:
f = h5py.File(h5_filename, 'a')
remove_if_exists(f, 'unmutated_a2d2')
f.close()
create_unmutated_dataset()

In [ ]:
f = h5py.File(h5_filename, 'a')
remove_if_exists(f, 'mutated_a2d2_coffee_10')
remove_if_exists(f, 'mutated_a2d2_coffee_label_10')
f.close()
create_mutated_dataset(mutation_probability=0.1, label="mutated_a2d2_coffee_10")

In [ ]:
f = h5py.File(h5_filename, 'a')
remove_if_exists(f, 'mutated_a2d2_coffee_100')
remove_if_exists(f, 'mutated_a2d2_coffee_label_100')
f.close()
create_fully_mutated_dataset()

In [ ]:
f = h5py.File(h5_filename, 'a')
remove_if_exists(f, 'white_coffee')
remove_if_exists(f, 'white_coffee_label')
f.close()
create_white_background_coffeemachines()
